In [33]:
import h5py
from Bio import SeqIO, AlignIO
import io
import numpy as np

pfam_path = '/home/cactuskid/Downloads/Pfam-A.full.gz'
import gzip

def yeild_alns(pfam_path , verbose = False):    
    with gzip.open(pfam_path, 'rb') as f:
        aln= ''
        acc = None
        for i,l in enumerate( f ):
            if verbose == True:
                if i < 1000:
                    print(l)
            aln+=l.decode()
            if acc is None and 'AC' in l.decode():
                acc = l.decode().split()[2]
            if l == b'//\n':
                yield acc, AlignIO.read(io.StringIO(aln), "stockholm")
                aln = ''
                acc = None

            
aln =  next(yeild_alns(pfam_path))
print(dir(aln))
print(aln) 

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count', 'index']
('PF00244', <<class 'Bio.Align.MultipleSeqAlignment'> instance (215 records of length 339, SingleLetterAlphabet()) at 7f4a9e9bff28>)


In [37]:
with h5py.File(pfam_path +'.h5', 'w') as hf:
    for pfam_id, msa in yeild_alns( pfam_path):
        align_array = np.array([ list(rec.upper())  for rec in msa], np.character)
        hf.create_dataset(pfam_id,  data=align_array)
    

KeyboardInterrupt: 